In [2]:
# Without modification for combined categories and adjusted loss
# on dev set only
# random crop not turned off yet
# haven't check logic for day/night yet

from __future__ import print_function, division
import argparse
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from torchvision import datasets, models, transforms

from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image, ImageFile
from tqdm import tqdm

from WildAnimalDataset import WildAnimalDataset

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
ImageFile.LOAD_TRUNCATED_IMAGES = True

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=16, help='training batch size')
#parser.add_argument('--photo_path', type=str, default='/data/cs341-bucket/camera_B2', help='photo path')
#parser.add_argument('--meta_path', type=str, default='/data/cs341-bucket/camera_B2/camera_B2.xlsx', help='metadata path')

parser.add_argument('--photo_path', type=str, default='/data/data_combined_final/dev_signs', help='photo path')
parser.add_argument('--meta_path', type=str, default='/data/metadata/combined_final_dev.xlsx', help='metadata path')


def obtain_inputs_parameters(args):

    meta_path = args.meta_path
    photo_path = args.photo_path

    data_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.31293, 0.30621,0.28194], [0.18229829,0.18250624,0.1759812])
    ])
        
    animal_dataset = WildAnimalDataset(meta_path, photo_path, data_transform) 
    dataloader = DataLoader(animal_dataset, args.batch_size, shuffle=False, num_workers=4)
    return dataloader

def read_trained_single_model(args, use_gpu, n_total_categories):
        
    model = models.densenet161(pretrained = True)
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, n_total_categories)
    model.train(False)
    
    if use_gpu:
        model = model.cuda()
    model_checkpoint_path = 'best_model.pt'
    print("model_checkpoint_path: " + model_checkpoint_path)
    model.load_state_dict(torch.load(model_checkpoint_path))
    return model

def create_single_model_confusion(model, n_total_categories, dataloader, use_gpu, args):
    
    confusion = torch.zeros(n_total_categories, n_total_categories)
    model.train(False) 

    for data in tqdm(dataloader):
        inputs, labels = data 
        labels = labels.view(-1)
        
        if use_gpu:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        outputs = model(inputs) 
        _, preds = torch.max(outputs.data, 1)

        for i, j in zip(preds, labels.data):
            confusion[i,j] += 1
    return confusion

# functions checked until this point
def write_confusion_csv(confusion, class_names, args):
    confusion_df = pd.DataFrame(confusion.numpy(), columns = class_names)
    confusion_output_name = "confusion_matrix/" + args.period + "_" + args.prob + "_" + args.model
    if args.norm == 1:
        confusion_output_name += "_normUp"
    if args.norm == 3:
        confusion_output_name += "_norm3"
    if args.optimizer == "Adam":
        confusion_output_name += "_Adam"
    if args.optimizer == "Nesterov":
        confusion_output_name += "_Nesterov"
    if args.model == "inceptionv3":
        confusion_output_name += "_lrdecay2"
    if args.norm == 2:
        confusion_output_name += "_normMeanImage_lrdecay"
    if args.lrdecay == 1:
        confusion_output_name += '_lrdecay'
    if args.sample_list != None:
        confusion_output_name += '_oslist'+''.join(str(e) for e in args.sample_list)
    if args.sample_N != 0:
        confusion_output_name += '_oversample' + str(args.sample_N)
        
    confusion_output_name += '.csv'
    confusion_df.to_csv(confusion_output_name, index = False)


def model_eval(confusion, n_total_categories, class_names):
    full_info = [] # [(species, recall, precision, f1_score), (species, recall, precision, f1_score), ...]
    for class_index in range(n_total_categories):
        recall = calculateRecall(class_index, confusion)
        precision = calculatePrecision(class_index, confusion)
        f1_score = calculateF1score(recall, precision)
        value = tuple([class_names[class_index], recall, precision, f1_score])
        full_info.append(value)
    
    model_eval_result = pd.DataFrame.from_records(full_info, columns = ['species', 'recall', 'precision', 'f1score'])
    return model_eval_result   

def write_metrics_to_csv(model_eval_result, args):
    eval_result_output_path = "evaluation_metrics/" + args.period + "_" + args.prob + "_" + args.model
    if args.norm == 1:
        eval_result_output_path += "_normUp"
    if args.norm == 3:
        eval_result_output_path += "_norm3"
    if args.optimizer == "Adam":
        eval_result_output_path += "_Adam"
    if args.optimizer == "Nesterov":
        eval_result_output_path += "_Nesterov"
    if args.model == "inceptionv3":
        eval_result_output_path += "_lrdecay2"
    if args.norm == 2:
        eval_result_output_path += "_normMeanImage_lrdecay"
    if args.lrdecay == 1:
        eval_result_output_path += '_lrdecay'
    if args.sample_list != None:
        eval_result_output_path += '_oslist'+''.join(str(e) for e in args.sample_list)
    if args.sample_N != 0:
        eval_result_output_path += '_oversample' + str(args.sample_N)
        
    eval_result_output_path += '.csv'
    model_eval_result.to_csv(eval_result_output_path, index = False)


def main():
    args = parser.parse_args()
    n_total_categories = 24
    use_gpu = True
    class_names = ["None", "Black-tailed deer", "Human", "Black-tailed Jackrabbit", "Coyote", 
                   "Big Category", "Brush Rabbit", "Western scrub-jay","Bobcat", "blank1","blank2", 
                   "California quail","Raccoon", "Mountain lion","Striped skunk", "Wild turkey", 
                   "Gray Fox", "Virginia Opossum", "Stellers jay", "Western Gray Squirrel", 
                   "Dusky-footed Woodrat", "Great blue heron", "Fox Squirrel", "California Ground Squirrel"]
    
    dataloader = obtain_inputs_parameters(args)
    model = read_trained_single_model(args, use_gpu, n_total_categories)
    
    write_pred(args)
    
    
    
    #confusion = create_single_model_confusion(model, n_total_categories, dataloader, use_gpu, args)
    #print("Total Accuracy: " + str(torch.sum(confusion.diag()) / confusion.sum())) 

#-------------------------------

In [7]:
def write_pred(args):
    meta = pd.io.excel.read_excel(args.meta_path)


In [9]:
main()

usage: ipykernel_launcher.py [-h] [--batch_size BATCH_SIZE]
                             [--photo_path PHOTO_PATH] [--meta_path META_PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-2cf9f8ce-58d0-4ee0-a410-c14c990ae86b.json


SystemExit: 2

In [2]:
import pandas as pd
meta = pd.io.excel.read_excel('/data/metadata/combined_final_dev.xlsx')

In [3]:
pd.DataFrame(columns = list(meta.columns.values) + ['pred'])

,X__1,SpecId,MetaId,Species,SpeciesOther,Comment,Age,Sex,UpdatedByName,UpdatedBySuid,...,CameraPath,CameraType,Light,SerialNum,Tagged,Upload,up_name_old,Contain,up_name,pred


In [88]:
len(meta)

11487

In [15]:
meta['pred'] = pred

In [16]:
meta.head()

,X__1,SpecId,MetaId,Species,SpeciesOther,Comment,Age,Sex,UpdatedByName,UpdatedBySuid,...,CameraPath,CameraType,Light,SerialNum,Tagged,Upload,up_name_old,Contain,up_name,pred
0,28,201,39185,0,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,...,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,204,R016WX,yes,2016-04-01,camera_B2_28.jpg,0,day_camera_B2_28.jpg,0.0
1,47,368805,382347,10,NaN,NaN,NaN,NaN,Robert Dodge,robert21,...,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,4,R016WX,yes,2017-12-19,camera_B2_47.jpg,1,day_camera_B2_47.jpg,0.0
2,50,165,39149,2,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,...,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,293,R016WX,yes,2016-04-01,camera_B2_50.jpg,1,day_camera_B2_50.jpg,0.0
3,54,405,39389,1,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,...,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,283,R016WX,yes,2016-04-01,camera_B2_54.jpg,1,day_camera_B2_54.jpg,0.0
4,126,257,39241,2,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,...,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,1724,R016WX,yes,2016-04-01,camera_B2_126.jpg,1,day_camera_B2_126.jpg,0.0


In [76]:
x = meta.iloc[1]

In [77]:
x

X__1                                       47
SpecId                                 368805
MetaId                                 382347
Species                                    10
SpeciesOther                              NaN
Comment                                   NaN
Age                                       NaN
Sex                                       NaN
UpdatedByName                    Robert Dodge
UpdatedBySuid                        robert21
UpdatedDateTime           2018-02-26 11:02:03
CorrectedByName                           NaN
CorrectedBySuid                           NaN
CorrectedDateTime                         NaT
Id                                     382347
Name                 p_000705.jpg20171114.jpg
EventDateTime             2017-11-14 06:40:54
MySQLDateTime             2017-11-14 06:40:54
EventTempC                                  0
CameraPath           CAMERA_ARRAY_B/camera_B2
CameraType                       BuckEyeCamII
Light                             

In [64]:
x['pred'] = 1

In [54]:
df = pd.DataFrame(columns = list(x.columns.values) + ['pred'])

In [59]:
df.loc[0] = 0

In [65]:
df.loc[0] = x

In [66]:
df

,X__1,SpecId,MetaId,Species,SpeciesOther,Comment,Age,Sex,UpdatedByName,UpdatedBySuid,...,CameraPath,CameraType,Light,SerialNum,Tagged,Upload,up_name_old,Contain,up_name,pred
0,47,368805,382347,10,NaN,NaN,NaN,NaN,Robert Dodge,robert21,...,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,4,R016WX,yes,2017-12-19 00:00:00,camera_B2_47.jpg,1,day_camera_B2_47.jpg,1


In [5]:
pd.read_csv('settled.csv')

,SpecId,MetaId,Species,SpeciesOther,Comment,Age,Sex,UpdatedByName,UpdatedBySuid,UpdatedDateTime,...,EventDateTime,MySQLDateTime,EventTempC,CameraPath,CameraType,Light,SerialNum,Tagged,Upload,pred
0,5,38989,Black-tailed deer,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 08:04:21,...,2010-08-20 03:41:16,2010-08-20 03:41:16,13,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,0,R016WX,yes,2016-04-01,Black-tailed deer
1,7,38991,Black-tailed deer,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 08:04:40,...,2010-08-27 02:23:19,2010-08-27 02:23:19,12,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,0,R016WX,yes,2016-04-01,Black-tailed deer
2,9,38993,Black-tailed deer,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 08:04:34,...,2010-08-27 08:28:07,2010-08-27 08:28:07,10,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,535,R016WX,yes,2016-04-01,Black-tailed deer
3,11,38995,Black-tailed deer,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 08:04:00,...,2010-08-31 07:20:10,2010-08-31 07:20:10,12,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,159,R016WX,yes,2016-04-01,Black-tailed deer
4,13,38997,Black-tailed deer,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 08:04:46,...,2010-10-01 06:58:24,2010-10-01 06:58:24,15,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,0,R016WX,yes,2016-04-01,Black-tailed deer
5,15,38999,Black-tailed deer,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 08:04:15,...,2010-10-04 07:06:06,2010-10-04 07:06:06,12,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,5,R016WX,yes,2016-04-01,Black-tailed deer
6,17,39001,Black-tailed deer,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 08:04:51,...,2010-10-10 04:01:12,2010-10-10 04:01:12,10,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,0,R016WX,yes,2016-04-01,Black-tailed deer
7,19,39003,Human,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 09:04:24,...,2010-10-11 10:25:54,2010-10-11 10:25:54,26,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,11173,R016WX,yes,2016-04-01,Human
8,21,39005,Human,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 09:04:08,...,2010-10-30 08:30:08,2010-10-30 08:30:08,10,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,500,R016WX,yes,2016-04-01,Human
9,23,39007,Human,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 09:04:28,...,2010-10-30 09:01:54,2010-10-30 09:01:54,10,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,751,R016WX,yes,2016-04-01,Human


In [4]:
pd.read_csv('unsettled.csv')

,SpecId,MetaId,Species,SpeciesOther,Comment,Age,Sex,UpdatedByName,UpdatedBySuid,UpdatedDateTime,...,EventDateTime,MySQLDateTime,EventTempC,CameraPath,CameraType,Light,SerialNum,Tagged,Upload,pred
0,285,39269,Coyote,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 10:04:03,...,2013-06-04 10:57:37,2013-06-04 10:57:37,20,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,2061,R016WX,yes,2016-04-01,Coyote
1,467,39451,None,NaN,NaN,NaN,NaN,Trevor Hebert,thebert,2016-04-25 10:04:52,...,2015-12-18 23:48:36,2015-12-18 23:48:36,10,CAMERA_ARRAY_B/camera_B2,BuckEyeCamII,0,R016WX,yes,2016-04-01,None


In [87]:
10979 + 508

11487

In [1]:
meta = '/data/cs341-bucket/camera_B2/camera_B2.xlsx'
test = meta.split('/')

In [3]:
test[-2]

'camera_B2'